In [1]:
import tensorflow as tf
import numpy as np
import keras
import time
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from callbacks import all_callbacks

%matplotlib inline
seed = 0
np.random.seed(seed)

2023-07-07 01:17:57.174409: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

cifar10_classes=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, MaxPooling2D, Flatten, Activation
from qkeras import QActivation, QDense, QConv2D
from qkeras.quantizers import quantized_bits, quantized_relu
from tensorflow.keras.regularizers import l1

In [16]:
model = Sequential()

model.add(
    Input(shape=(32, 32, 3))
)

# Conv 1
model.add(
    QConv2D(
    5,
    kernel_size=(5, 5),
    strides=(1, 1),
    padding='valid',
    name='conv1',
    kernel_quantizer=quantized_bits(16, 6, alpha=1),
    bias_quantizer=quantized_bits(16, 6, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(6), name='relu1')
)
model.add(
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='pool1')
)

# Conv 2
model.add(
    QConv2D(
    13,
    kernel_size=(5, 5),
    strides=(1, 1),
    padding='valid',
    name='conv2',
    kernel_quantizer=quantized_bits(16, 6, alpha=1),
    bias_quantizer=quantized_bits(16, 6, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(6), name='relu2')
)
model.add(
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='pool2')
)

# Conv 3
model.add(
    QConv2D(
    20,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv3',
    kernel_quantizer=quantized_bits(16, 6, alpha=1),
    bias_quantizer=quantized_bits(16, 6, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(6), name='relu3')
)

# Conv 4
model.add(
    QConv2D(
    20,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv4',
    kernel_quantizer=quantized_bits(16, 6, alpha=1),
    bias_quantizer=quantized_bits(16, 6, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(6), name='relu4')
)

# Conv 5
model.add(
    QConv2D(
    13,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv5',
    kernel_quantizer=quantized_bits(16, 6, alpha=1),
    bias_quantizer=quantized_bits(16, 6, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(6), name='relu5')
)
model.add(
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='pool5')
)

# Flatten
model.add(Flatten())

# FC 1
model.add(
    QDense(
        214,
        name='fc1',
        kernel_quantizer=quantized_bits(16, 6, alpha=1),
        bias_quantizer=quantized_bits(16, 6, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(6), name='relu6')
)

# FC 2
model.add(
    QDense(
        16,
        name='fc2',
        kernel_quantizer=quantized_bits(16, 6, alpha=1),
        bias_quantizer=quantized_bits(16, 6, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(6), name='relu7')
)

# FC 3
model.add(
    QDense(
        10,
        name='output',
        kernel_quantizer=quantized_bits(16, 6, alpha=1),
        bias_quantizer=quantized_bits(16, 6, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)

model.add(Activation(activation='softmax', name='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (QConv2D)             (None, 28, 28, 5)         380       
                                                                 
 relu1 (QActivation)         (None, 28, 28, 5)         0         
                                                                 
 pool1 (MaxPooling2D)        (None, 13, 13, 5)         0         
                                                                 
 conv2 (QConv2D)             (None, 9, 9, 13)          1638      
                                                                 
 relu2 (QActivation)         (None, 9, 9, 13)          0         
                                                                 
 pool2 (MaxPooling2D)        (None, 4, 4, 13)          0         
                                                                 
 conv3 (QConv2D)             (None, 4, 4, 20)         

In [17]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(0.75, begin_step=2000, frequency=100)}
model = prune.prune_low_magnitude(model, **pruning_params)

In [19]:
train = True
if train:
    adam = Adam(learning_rate=0.001)
    model.compile(
        optimizer=adam, 
        loss=['categorical_crossentropy'], 
        metrics=['accuracy'])
    
    model.fit(
        x_train,
        y_train,
        batch_size=4096,
        epochs=500,
        validation_data=(x_test, y_test),
        shuffle=True,
    )
    
    model = strip_pruning(model)
    model.save('AlexNet_Weights/AlexNet_check_best_model.h5')
else:
    from tensorflow.keras.models import load_model
    from qkeras.utils import _add_supported_quantized_objects

    co = {}
    _add_supported_quantized_objects(co)
    model = load_model('AlexNet_Weights/AlexNet_check_best_model.h5', custom_objects=co)

Epoch 1/500
13/13 [==============================] - 11s 483ms/step - loss: 2.4532 - accuracy: 0.1321 - val_loss: 2.4227 - val_accuracy: 0.1768
Epoch 2/500
13/13 [==============================] - 5s 402ms/step - loss: 2.3810 - accuracy: 0.1880 - val_loss: 2.3280 - val_accuracy: 0.1848
Epoch 3/500
13/13 [==============================] - 5s 404ms/step - loss: 2.3126 - accuracy: 0.1697 - val_loss: 2.3013 - val_accuracy: 0.1623
Epoch 4/500
13/13 [==============================] - 5s 407ms/step - loss: 2.2762 - accuracy: 0.1856 - val_loss: 2.2497 - val_accuracy: 0.2144
Epoch 5/500
13/13 [==============================] - 5s 404ms/step - loss: 2.2311 - accuracy: 0.2188 - val_loss: 2.2073 - val_accuracy: 0.2235
Epoch 6/500
13/13 [==============================] - 5s 405ms/step - loss: 2.1858 - accuracy: 0.2387 - val_loss: 2.1856 - val_accuracy: 0.2485
Epoch 7/500
13/13 [==============================] - 5s 406ms/step - loss: 2.1523 - accuracy: 0.2656 - val_loss: 2.1308 - val_accuracy: 0.280


Epoch 00311: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 312/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1888 - accuracy: 0.5906 - val_loss: 1.2842 - val_accuracy: 0.5558

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00312: val_loss did not improve from 1.25042

Epoch 00312: val_loss did not improve from 1.25042

Epoch 00312: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00312: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 313/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1919 - accuracy: 0.5896 - val_loss: 1.2678 - val_accuracy: 0.5564

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00313: val_loss did not improve from 1.25042

Epoch 00313: val_loss did not improve from 1.25042

Epoch 00313: saving model to AlexNet_Weights/Qkeras/KERAS_check_mode


Epoch 00327: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 328/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2039 - accuracy: 0.5849 - val_loss: 1.2531 - val_accuracy: 0.5616

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00328: val_loss did not improve from 1.24756

Epoch 00328: val_loss did not improve from 1.24756

Epoch 00328: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00328: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 329/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1953 - accuracy: 0.5890 - val_loss: 1.2593 - val_accuracy: 0.5598

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00329: val_loss did not improve from 1.24756

Epoch 00329: val_loss did not improve from 1.24756

Epoch 00329: saving model to AlexNet_Weights/Qkeras/KERAS_check_mode


Epoch 00343: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 344/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1881 - accuracy: 0.5905 - val_loss: 1.2795 - val_accuracy: 0.5547

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00344: val_loss did not improve from 1.24756

Epoch 00344: val_loss did not improve from 1.24756

Epoch 00344: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00344: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 345/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2012 - accuracy: 0.5873 - val_loss: 1.2791 - val_accuracy: 0.5579

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00345: val_loss did not improve from 1.24756

Epoch 00345: val_loss did not improve from 1.24756

Epoch 00345: saving model to AlexNet_Weights/Qkeras/KERAS_check_mode

49/49 [==============================] - 1s 19ms/step - loss: 1.1915 - accuracy: 0.5900 - val_loss: 1.2493 - val_accuracy: 0.5657

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00360: val_loss did not improve from 1.24756

Epoch 00360: val_loss did not improve from 1.24756

Epoch 00360: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00360: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

Epoch 00360: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_epoch360.h5

***callbacks end***

Epoch 361/5000
49/49 [==============================] - 1s 20ms/step - loss: 1.1911 - accuracy: 0.5893 - val_loss: 1.2579 - val_accuracy: 0.5626

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00361: val_loss did not improve from 1.24756

Epoch 00361: val_loss did not improve from 1.24756

Epoch 00361: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00361: saving model to A

...

49/49 [==============================] - 1s 19ms/step - loss: 1.1865 - accuracy: 0.5906 - val_loss: 1.2702 - val_accuracy: 0.5590

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00440: val_loss did not improve from 1.24756

Epoch 00440: val_loss did not improve from 1.24756

Epoch 00440: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00440: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

Epoch 00440: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_epoch440.h5

***callbacks end***

Epoch 441/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2021 - accuracy: 0.5832 - val_loss: 1.2719 - val_accuracy: 0.5552

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00441: val_loss did not improve from 1.24756

Epoch 00441: val_loss did not improve from 1.24756

Epoch 00441: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00441: saving model to A


***callbacks end***

Epoch 456/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2028 - accuracy: 0.5859 - val_loss: 1.2973 - val_accuracy: 0.5492

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00456: val_loss did not improve from 1.24756

Epoch 00456: val_loss did not improve from 1.24756

Epoch 00456: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00456: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 457/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1968 - accuracy: 0.5867 - val_loss: 1.2571 - val_accuracy: 0.5620

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00457: val_loss did not improve from 1.24756

Epoch 00457: val_loss did not improve from 1.24756

Epoch 00457: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00457: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_w


***callbacks end***

Epoch 472/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1821 - accuracy: 0.5937 - val_loss: 1.2782 - val_accuracy: 0.5547

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00472: val_loss did not improve from 1.24756

Epoch 00472: val_loss did not improve from 1.24756

Epoch 00472: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00472: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 473/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1840 - accuracy: 0.5918 - val_loss: 1.2867 - val_accuracy: 0.5524

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00473: val_loss did not improve from 1.24756

Epoch 00473: val_loss did not improve from 1.24756

Epoch 00473: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00473: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_w

49/49 [==============================] - 1s 19ms/step - loss: 1.1838 - accuracy: 0.5919 - val_loss: 1.2540 - val_accuracy: 0.5625

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00488: val_loss did not improve from 1.24756

Epoch 00488: val_loss did not improve from 1.24756

Epoch 00488: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00488: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 489/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1911 - accuracy: 0.5907 - val_loss: 1.2702 - val_accuracy: 0.5588

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00489: val_loss did not improve from 1.24756

Epoch 00489: val_loss did not improve from 1.24756

Epoch 00489: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00489: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch


***callbacks end***

Epoch 504/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1937 - accuracy: 0.5890 - val_loss: 1.2611 - val_accuracy: 0.5609

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00504: val_loss did not improve from 1.24756

Epoch 00504: val_loss did not improve from 1.24756

Epoch 00504: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00504: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 505/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1883 - accuracy: 0.5891 - val_loss: 1.2524 - val_accuracy: 0.5650

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00505: val_loss did not improve from 1.24756

Epoch 00505: val_loss did not improve from 1.24756

Epoch 00505: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00505: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_w